In [1]:
# Parâmetros do pipeline
input_config_path = "config.json"  # Caminho do arquivo de configuração
input_metricas_path = "metricas_completas.csv"  # Arquivo de métricas calculadas
bucket_name = "experimento-lucas-barbosa"  # Nome do bucket S3
pasta_processed = "processed/"  # Pasta para arquivos processados
pasta_logs = "logs/"  # Pasta para logs de execução

In [2]:
import pandas as pd
import boto3
import json
import os
from datetime import datetime

In [3]:
print(f"Início do salvamento: {datetime.now()}")
print(f"Configuração: {input_config_path}")
print(f"Métricas: {input_metricas_path}")
print(f"Bucket: {bucket_name}")

Início do salvamento: 2025-07-20 22:09:00.956218
Configuração: config.json
Métricas: metricas_completas.csv
Bucket: experimento-lucas-barbosa


In [4]:
# Carregar configurações e resultados
with open(input_config_path) as f:
    config = json.load(f)

# Usar parâmetros do pipeline ou valores do config
BUCKET = bucket_name or config.get('bucket_name')
RAW_PREF = config['pasta_raw']
PROC_PREF = pasta_processed or config.get('pasta_processed')
ARCH_PREF = config['pasta_archive']
LOGS_PREF = pasta_logs

print("Configurações carregadas:")
print(f"  Bucket: {BUCKET}")
print(f"  Pasta raw: {RAW_PREF}")
print(f"  Pasta processed: {PROC_PREF}")
print(f"  Pasta archive: {ARCH_PREF}")
print(f"  Pasta logs: {LOGS_PREF}")

Configurações carregadas:
  Bucket: experimento-lucas-barbosa
  Pasta raw: raw/
  Pasta processed: processed/
  Pasta archive: archive/
  Pasta logs: logs/


In [5]:
# Carregar métricas completas
try:
    df_meta = pd.read_csv(input_metricas_path)
    print(f"✓ Métricas carregadas: {len(df_meta)} linhas")
    print(f"  Colunas: {list(df_meta.columns)}")
    
    # Verificar se há dados
    if len(df_meta) == 0:
        raise ValueError("Arquivo de métricas está vazio")
        
    # Mostrar resumo das métricas
    print(f"  Experimentos: {df_meta['id_experimento'].nunique()}")
    print(f"  Grupos: {df_meta['grupo'].nunique()}")
    print(f"  Métricas: {df_meta['metrica'].nunique()}")
    
except Exception as e:
    print(f"❌ Erro ao carregar métricas: {str(e)}")
    raise

✓ Métricas carregadas: 84 linhas
  Colunas: ['id_experimento', 'grupo', 'metrica', 'valor', 'timestamp', 'arquivo_origem']
  Experimentos: 5
  Grupos: 3
  Métricas: 8


In [6]:
# Extrair ID do experimento (único ou usar o primeiro)
id_experimento = df_meta['id_experimento'].iloc[0]
print(f"ID do experimento: {id_experimento}")

# Conectar S3
s3 = boto3.client('s3')
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Gerar nomes seguindo o padrão
nome_meta = f"{id_experimento}_metricas_{timestamp}.csv"
key_meta = f"{PROC_PREF}{nome_meta}"

print(f"Arquivo de saída: {nome_meta}")
print(f"Chave S3: {key_meta}")

ID do experimento: exp_005
Arquivo de saída: exp_005_metricas_20250720_220901.csv
Chave S3: processed/exp_005_metricas_20250720_220901.csv


In [7]:
# Conectar S3 e upload métricas
s3 = boto3.client('s3')
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Gerar nome seguindo o padrão
nome_meta = f"{id_experimento}_metricas_{timestamp}.csv"
key_meta  = f"{PROC_PREF}{nome_meta}"

# Salvar localmente e enviar
df_meta.to_csv(f"/tmp/{nome_meta}", index=False)
s3.upload_file(f"/tmp/{nome_meta}", BUCKET, key_meta)
print(f"✓ Métricas enviadas para s3://{BUCKET}/{key_meta}")

✓ Métricas enviadas para s3://experimento-lucas-barbosa/processed/exp_005_metricas_20250720_220901.csv


In [8]:
# Salvar log de execução
try:
    log = {
        'timestamp': datetime.now().isoformat(),
        'id_experimento': id_experimento,
        'metrics_key': key_meta,
        'metrics_file': nome_meta,
        'total_linhas': len(df_meta),
        'grupos': list(df_meta['grupo'].unique()),
        'metricas': list(df_meta['metrica'].unique()),
        'bucket': BUCKET,
        'status': 'sucesso'
    }
    
    log_key = f"{LOGS_PREF}processamento_{id_experimento}_{timestamp}.json"
    
    # Upload do log
    s3.put_object(
        Bucket=BUCKET, 
        Key=log_key, 
        Body=json.dumps(log, indent=2, ensure_ascii=False),
        ContentType='application/json'
    )
    
    print(f"✓ Log salvo em s3://{BUCKET}/{log_key}")
    
except Exception as e:
    print(f"⚠️ Erro ao salvar log: {str(e)}")
    # Não falha o pipeline se o log der erro

✓ Log salvo em s3://experimento-lucas-barbosa/logs/processamento_exp_005_20250720_220901.json
